In [ ]:
import pandas as pd
import numpy as np
import os

# Hàm split_dataframe giữ nguyên như cũ
def split_dataframe(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Nhận một DataFrame, tách nó thành hai phần (đúng và cần crawl lại)
    dựa trên các điều kiện đã xác định và trả về hai DataFrame đã được
    loại bỏ trùng lặp.
    """
    df_copy = df.copy()

    main_link_correct = df_copy['MainLinkCorrect'].fillna('').astype(str).str.strip().str.upper()
    date_correct = df_copy['ConferenceDateCorrect'].fillna('').astype(str).str.strip().str.upper()
    location_correct = df_copy['LocationCorrect'].fillna('').astype(str).str.strip().str.upper()
    type_correct = df_copy['TypeCorrect'].fillna('').astype(str).str.strip().str.upper()

    condition1 = (
        (main_link_correct == 'FALSE') |
        (date_correct == 'FALSE') |
        (location_correct == 'FALSE') |
        (type_correct == 'FALSE')
    )

    percent_cols = ['Percent', 'Percent_1', 'Percent_2', 'Percent_3', 'Percent_4']
    for col in percent_cols:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

    condition2_parts = [df_copy[col].notna() & (df_copy[col] != 100) for col in percent_cols]
    condition2 = False
    for part in condition2_parts:
        condition2 = condition2 | part

    recrawl_condition = condition1 | condition2
    recrawl_df = df_copy[recrawl_condition]
    correct_df = df_copy[~recrawl_condition]

    subset_cols = ['title', 'acronym']
    unique_recrawl_df = recrawl_df.drop_duplicates(subset=subset_cols, keep='first')
    unique_correct_df = correct_df.drop_duplicates(subset=subset_cols, keep='first')

    return unique_correct_df, unique_recrawl_df


# ==============================================================================
# HÀM process_file_list ĐÃ ĐƯỢC CẬP NHẬT
# ==============================================================================
def process_file_list(
    input_files: list,
    output_dir: str,
    aggregated_correct_path: str,
    aggregated_recrawl_path: str,
    recrawled_files: list = None,
    definitive_correct_path: str = None  # <-- THAM SỐ MỚI
):
    """
    Xử lý một danh sách các file CSV, đối chiếu với kết quả đã recrawl,
    và tạo ra hai file tổng hợp cuối cùng.
    Sử dụng file definitive_correct_path làm bộ lọc cuối cùng cho file recrawl.
    """
    # --- CÁC BƯỚC 1, 2, 3, 4 GIỮ NGUYÊN NHƯ TRƯỚC ---
    # (Code từ bước 1 đến 4 được rút gọn ở đây để dễ đọc, bạn chỉ cần copy cả hàm)
    all_correct_dfs = []
    all_recrawl_dfs = []
    os.makedirs(output_dir, exist_ok=True)
    # --- BƯỚC 1: XỬ LÝ CÁC FILE INPUT BAN ĐẦU ---
    for file_path in input_files:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str)
        except Exception as e:
            print(f"Lỗi đọc file {file_path}: {e}")
            continue
        correct_df, recrawl_df = split_dataframe(df)
        all_correct_dfs.append(correct_df)
        all_recrawl_dfs.append(recrawl_df)
    # --- BƯỚC 2: GỘP KẾT QUẢ ---
    master_correct_df = pd.concat(all_correct_dfs, ignore_index=True) if all_correct_dfs else pd.DataFrame()
    master_recrawl_df = pd.concat(all_recrawl_dfs, ignore_index=True) if all_recrawl_dfs else pd.DataFrame()
    # --- BƯỚC 3: ĐỐI CHIẾU VỚI CÁC FILE ĐÃ RECRAWL ---
    if recrawled_files:
        recrawled_dfs_list = []
        for r_file in recrawled_files:
            try:
                df_recrawled = pd.read_csv(r_file, encoding='utf-8-sig', na_values=['', ' '], dtype=str)
                recrawled_dfs_list.append(df_recrawled)
            except Exception as e:
                print(f"Lỗi đọc file recrawl {r_file}: {e}")
        if recrawled_dfs_list:
            recrawled_master_df = pd.concat(recrawled_dfs_list, ignore_index=True)
            newly_correct_df_from_recrawl, _ = split_dataframe(recrawled_master_df)
            if not newly_correct_df_from_recrawl.empty:
                newly_correct_df_from_recrawl['title'] = newly_correct_df_from_recrawl['title'].str.strip()
                newly_correct_df_from_recrawl['acronym'] = newly_correct_df_from_recrawl['acronym'].astype(str).str.strip()
                corrected_keys = set(tuple(x) for x in newly_correct_df_from_recrawl[['title', 'acronym']].values)
                
                master_recrawl_df['title_stripped'] = master_recrawl_df['title'].str.strip()
                master_recrawl_df['acronym_stripped'] = master_recrawl_df['acronym'].astype(str).str.strip()
                mask_to_remove = master_recrawl_df.apply(lambda row: (row['title_stripped'], row['acronym_stripped']) in corrected_keys, axis=1)
                
                master_recrawl_df = master_recrawl_df[~mask_to_remove].copy()
                master_correct_df = pd.concat([master_correct_df, newly_correct_df_from_recrawl], ignore_index=True)
                master_recrawl_df.drop(columns=['title_stripped', 'acronym_stripped'], inplace=True, errors='ignore')
    # --- BƯỚC 4: LỌC CUỐI CÙNG DỰA TRÊN CỘT 'Note' ---
    if 'Note' in master_correct_df.columns:
        note_exists_condition_final = master_correct_df['Note'].notna() & (master_correct_df['Note'].astype(str).str.strip() != '')
        rows_to_move_final = master_correct_df[note_exists_condition_final].copy()
        if not rows_to_move_final.empty:
            master_recrawl_df = pd.concat([master_recrawl_df, rows_to_move_final], ignore_index=True)
            master_correct_df = master_correct_df[~note_exists_condition_final].copy()

    # ==============================================================================
    # BƯỚC 4.5 (MỚI): LỌC FILE RECRAWL DỰA TRÊN FILE CORRECT THỦ CÔNG
    # ==============================================================================
    if definitive_correct_path and os.path.exists(definitive_correct_path):
        print(f"\n--- BƯỚC 4.5: Lọc danh sách recrawl dựa trên file '{definitive_correct_path}' ---")
        try:
            # Đọc file correct thủ công mà bạn cung cấp
            definitive_correct_df = pd.read_csv(definitive_correct_path, encoding='utf-8-sig', dtype=str).fillna('')

            # Tạo một tập hợp các khóa (title, acronym) từ file này để tra cứu nhanh
            definitive_correct_df['title'] = definitive_correct_df['title'].str.strip()
            definitive_correct_df['acronym'] = definitive_correct_df['acronym'].str.strip()
            keys_to_exclude = set(tuple(x) for x in definitive_correct_df[['title', 'acronym']].values)

            print(f"Đã xác định {len(keys_to_exclude)} dòng 'chắc chắn đúng' từ file thủ công.")

            if not master_recrawl_df.empty:
                initial_recrawl_count = len(master_recrawl_df)

                # Tạo các khóa tạm thời trong danh sách recrawl để so sánh
                master_recrawl_df['temp_title'] = master_recrawl_df['title'].astype(str).str.strip()
                master_recrawl_df['temp_acronym'] = master_recrawl_df['acronym'].astype(str).str.strip().fillna('')

                # Tạo một "mặt nạ" boolean để xác định những dòng cần XÓA
                # (những dòng có khóa nằm trong tập hợp keys_to_exclude)
                mask_to_remove = master_recrawl_df.apply(
                    lambda row: (row['temp_title'], row['temp_acronym']) in keys_to_exclude,
                    axis=1
                )

                # Áp dụng mặt nạ ngược (~) để GIỮ LẠI những dòng KHÔNG cần xóa
                master_recrawl_df = master_recrawl_df[~mask_to_remove].copy()

                # Dọn dẹp các cột tạm thời
                master_recrawl_df.drop(columns=['temp_title', 'temp_acronym'], inplace=True)

                final_recrawl_count = len(master_recrawl_df)
                print(f"Đã loại bỏ {initial_recrawl_count - final_recrawl_count} dòng khỏi danh sách recrawl vì chúng đã có trong file correct thủ công.")
            else:
                print("Danh sách recrawl đã rỗng, không cần lọc.")

        except FileNotFoundError:
            print(f"Cảnh báo: Không tìm thấy file correct thủ công '{definitive_correct_path}'. Bỏ qua bước lọc này.")
        except Exception as e:
            print(f"Lỗi khi xử lý file correct thủ công: {e}. Bỏ qua bước lọc này.")
    else:
        print("\nKhông có file correct thủ công nào được cung cấp, bỏ qua bước lọc 4.5.")


    # --- BƯỚC 5: LOẠI BỎ TRÙNG LẶP LẦN CUỐI VÀ LƯU FILE ---
    print("\n--- Hoàn tất và lưu file tổng hợp cuối cùng ---")

    # Xử lý và lưu file CORRECT tổng
    # Lưu ý: Script vẫn sẽ tạo ra file correct dựa trên logic của nó.
    # Bạn có thể bỏ qua file này và chỉ sử dụng file correct thủ công của mình.
    if not master_correct_df.empty:
        final_correct_df = master_correct_df.drop_duplicates(subset=['title', 'acronym'], keep='first')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu {len(final_correct_df)} dòng đúng (dựa trên logic script) vào '{aggregated_correct_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'đúng' nào theo logic script. Đã tạo file rỗng '{aggregated_correct_path}'")

    # Xử lý và lưu file RECRAWL tổng (đã được lọc ở bước 4.5)
    if not master_recrawl_df.empty:
        final_recrawl_df = master_recrawl_df.drop_duplicates(subset=['title', 'acronym'], keep='first')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng cần crawl lại (sau khi lọc) vào '{aggregated_recrawl_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'cần recrawl' nào. Đã tạo file rỗng '{aggregated_recrawl_path}'.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # 1. Danh sách các file input ban đầu
    input_csv_files = [
        './src/conference/evaluate/batch2.csv',
        './src/conference/evaluate/batch3.csv',
        './src/conference/evaluate/batch8.csv',
        './src/conference/evaluate/batch12.csv',
        './src/conference/evaluate/batch13.csv',
        './src/conference/evaluate/batch16.csv',
        './src/conference/evaluate/batch19.csv',
    ]

    # 2. Danh sách các file là kết quả của lần recrawl trước đó
    recrawled_results_files = [
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        
        # THÊM CÁC FILE RECRAWL MỚI VÀO ĐÂY
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',

        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',

    ]


    # 3. Thư mục để chứa các file output riêng lẻ (giữ nguyên)
    individual_output_directory = './src/conference/evaluate/individual_outputs'

    # 4. Đường dẫn cho 2 file tổng hợp cuối cùng (giữ nguyên)
    aggregated_correct_file = './src/conference/evaluate/ALL_BATCHES_correct_generated_by_script.csv'
    aggregated_recrawl_file = './src/conference/evaluate/ALL_BATCHES_recrawl_final.csv'

    # 5. (QUAN TRỌNG) Đường dẫn đến file correct thủ công của bạn
    # Đây là file "chân lý" mà bạn đã tạo
    definitive_correct_file_path = './src/conference/evaluate/ALL_BATCHES_correct_final.csv'

    # 6. Gọi hàm xử lý chính với tham số mới
    process_file_list(
        input_files=input_csv_files,
        output_dir=individual_output_directory,
        aggregated_correct_path=aggregated_correct_file,
        aggregated_recrawl_path=aggregated_recrawl_file,
        recrawled_files=recrawled_results_files,
        definitive_correct_path=definitive_correct_file_path # <-- TRUYỀN VÀO ĐÂY
    )

In [ ]:
import pandas as pd
import os

def split_df_by_note(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Tách một DataFrame thành hai phần dựa trên sự tồn tại của giá trị trong cột 'Note'.

    - Dữ liệu "cần recrawl": Các dòng có giá trị trong cột 'Note'.
    - Dữ liệu "đúng": Các dòng không có giá trị trong cột 'Note' (trống hoặc NaN).

    Args:
        df (pd.DataFrame): DataFrame đầu vào.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: Một tuple chứa (correct_df, recrawl_df).
    """
    # Kiểm tra an toàn để đảm bảo cột 'Note' tồn tại
    # Lưu ý: Sửa 'Note' thành 'note' nếu tên cột thực sự là chữ thường
    if 'note' not in df.columns: 
        print("Cảnh báo: Không tìm thấy cột 'note'. Coi như tất cả các dòng đều 'đúng'.")
        return df.copy(), pd.DataFrame(columns=df.columns)

    # Điều kiện: cột 'note' có giá trị (không phải NaN và không phải chuỗi rỗng sau khi strip)
    # Áp dụng .str.strip() trước khi so sánh
    condition_recrawl = df['note'].notna() & (df['note'].astype(str).str.strip() != '')

    recrawl_df = df[condition_recrawl].copy()
    correct_df = df[~condition_recrawl].copy()

    return correct_df, recrawl_df

def process_files_by_note(
    input_files: list, 
    output_dir: str,
    aggregated_correct_path: str, 
    aggregated_recrawl_path: str,
    recrawled_files: list = None # THÊM THAM SỐ MỚI
):
    """
    Xử lý một danh sách file, lọc từng file dựa trên cột 'Note'
    và chỉ xét những dòng có cột 'title' có giá trị.
    Sau đó đối chiếu với các file đã recrawl để loại bỏ những dòng đã được sửa đúng.
    Lưu output riêng lẻ và hai file tổng hợp cuối cùng.
    """
    all_correct_dfs = []
    all_recrawl_dfs = []

    os.makedirs(output_dir, exist_ok=True)

    # --- BƯỚC 1: XỬ LÝ CÁC FILE INPUT BAN ĐẦU ---
    for file_path in input_files:
        print(f"\n--- Đang xử lý file input: {file_path} ---")
        try:
            # Đọc file với dtype=str để đảm bảo cột 'note' và 'title' được đọc đúng dạng chuỗi
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
            print(f"Đã đọc thành công {len(df)} dòng.")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file '{file_path}'. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Đã xảy ra lỗi khi đọc file '{file_path}': {e}. Bỏ qua file này.")
            continue

        # Chỉ xét những dòng có cột 'title' có giá trị
        if 'title' in df.columns:
            initial_rows = len(df)
            df = df[df['title'].notna() & (df['title'].astype(str).str.strip() != '')].copy()
            if len(df) < initial_rows:
                print(f"Đã loại bỏ {initial_rows - len(df)} dòng do cột 'title' trống.")
        else:
            print("Cảnh báo: Không tìm thấy cột 'title'. Không thể lọc theo tiêu chí 'title có giá trị'.")
        
        if df.empty:
            print("Không còn dòng nào sau khi lọc theo 'title'. Bỏ qua file này.")
            continue

        correct_df, recrawl_df = split_df_by_note(df)
        
        print(f"Kết quả tách ban đầu: {len(correct_df)} dòng không có ghi chú, {len(recrawl_df)} dòng có ghi chú.")

        if not correct_df.empty:
            all_correct_dfs.append(correct_df)
        if not recrawl_df.empty:
            all_recrawl_dfs.append(recrawl_df)

        # --- Lưu các file output riêng lẻ ---
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        individual_correct_path = os.path.join(output_dir, f"{base_name}_note_correct.csv")
        individual_recrawl_path = os.path.join(output_dir, f"{base_name}_note_recrawl.csv")

        correct_df.to_csv(individual_correct_path, index=False, encoding='utf-8-sig')
        recrawl_df.to_csv(individual_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu output riêng lẻ vào thư mục '{output_dir}'")

    # --- BƯỚC 2: GỘP CÁC DATAFRAME BAN ĐẦU ---
    if not all_correct_dfs and not all_recrawl_dfs:
        print("\nKhông có dữ liệu từ các file input để xử lý. Kết thúc.")
        return

    print("\n--- Đang tổng hợp kết quả từ các file input ---")
    master_correct_df = pd.concat(all_correct_dfs, ignore_index=True) if all_correct_dfs else pd.DataFrame()
    master_recrawl_df = pd.concat(all_recrawl_dfs, ignore_index=True) if all_recrawl_dfs else pd.DataFrame()

    # --- BƯỚC 3 (MỚI): ĐỐI CHIẾU VỚI KẾT QUẢ ĐÃ RECRAWL DỰA TRÊN 'note' ---
    if recrawled_files:
        print("\n--- Đang đối chiếu với kết quả đã recrawl (dựa trên cột 'note') ---")
        recrawled_dfs_list = []
        for r_file in recrawled_files:
            try:
                # Đọc file đã recrawl, đảm bảo dtype=str cho 'note' và 'title'
                df = pd.read_csv(r_file, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
                # Lọc title cho file đã recrawl cũng
                if 'title' in df.columns:
                    df = df[df['title'].notna() & (df['title'].astype(str).str.strip() != '')].copy()
                else:
                    print(f"Cảnh báo: File đã recrawl '{r_file}' không có cột 'title'.")

                if not df.empty:
                    recrawled_dfs_list.append(df)
                    print(f"Đã đọc file đã recrawl: {r_file}")
                else:
                    print(f"File đã recrawl '{r_file}' trống sau khi lọc 'title'.")
            except FileNotFoundError:
                print(f"Cảnh báo: Không tìm thấy file đã recrawl '{r_file}'. Bỏ qua.")
            except Exception as e:
                print(f"Lỗi khi đọc file đã recrawl '{r_file}': {e}. Bỏ qua.")
        
        if recrawled_dfs_list:
            recrawled_master_df = pd.concat(recrawled_dfs_list, ignore_index=True)
            
            # Tái sử dụng split_df_by_note để xác định những dòng nào giờ đã "đúng" (không có note)
            newly_correct_after_recrawl_df, _ = split_df_by_note(recrawled_master_df)
            
            if not newly_correct_after_recrawl_df.empty:
                print(f"Tìm thấy {len(newly_correct_after_recrawl_df)} dòng đã không còn ghi chú sau khi recrawl.")
                
                # Chuẩn bị keys để đối chiếu
                # Đảm bảo title/acronym không trống trước khi tạo keys
                newly_correct_after_recrawl_df['title'] = newly_correct_after_recrawl_df['title'].str.strip()
                if 'acronym' in newly_correct_after_recrawl_df.columns:
                    newly_correct_after_recrawl_df['acronym'] = newly_correct_after_recrawl_df['acronym'].str.strip()

                subset_keys = ['title']
                if 'acronym' in newly_correct_after_recrawl_df.columns:
                    subset_keys.append('acronym')

                # Loại bỏ các dòng có title/acronym rỗng sau khi strip trước khi tạo keys
                newly_correct_after_recrawl_df = newly_correct_after_recrawl_df[newly_correct_after_recrawl_df['title'] != '']
                if 'acronym' in newly_correct_after_recrawl_df.columns:
                    newly_correct_after_recrawl_df = newly_correct_after_recrawl_df[newly_correct_after_recrawl_df['acronym'] != '']

                # Tạo set các khóa của những dòng đã được sửa đúng
                corrected_keys_after_recrawl = set(
                    tuple(x) for x in newly_correct_after_recrawl_df[subset_keys].values
                )
                
                # --- CẬP NHẬT master_recrawl_df và master_correct_df ---
                # 1. Loại bỏ các dòng đã được sửa đúng khỏi danh sách recrawl tổng
                # Tạo một Series với các khóa (title, acronym) của master_recrawl_df để so sánh
                master_recrawl_keys = master_recrawl_df[subset_keys].apply(tuple, axis=1)
                is_now_correct_mask = master_recrawl_keys.isin(corrected_keys_after_recrawl)
                
                # Giữ lại những dòng KHÔNG có trong danh sách đã sửa đúng
                master_recrawl_df_kept = master_recrawl_df[~is_now_correct_mask].copy()
                master_recrawl_df_moved = master_recrawl_df[is_now_correct_mask].copy() # Các dòng bị di chuyển
                
                master_recrawl_df = master_recrawl_df_kept
                print(f"Đã loại bỏ {len(master_recrawl_df_moved)} dòng khỏi danh sách recrawl tổng vì đã được sửa đúng.")

                # 2. Thêm các dòng vừa được sửa đúng vào danh sách correct tổng
                master_correct_df = pd.concat([master_correct_df, newly_correct_after_recrawl_df], ignore_index=True)
                print(f"Đã thêm {len(newly_correct_after_recrawl_df)} dòng vào danh sách đúng tổng.")
            else:
                print("Không có dòng nào được coi là 'đúng' trong các file đã recrawl (theo cột 'note').")
        else:
            print("Không có file đã recrawl nào hợp lệ để đối chiếu.")

    # --- BƯỚC 4: LOẠI BỎ TRÙNG LẶP LẦN CUỐI VÀ LƯU FILE ---
    print("\n--- Hoàn tất và lưu file tổng hợp cuối cùng ---")
    
    # Chuẩn bị cột cho drop_duplicates lần cuối
    subset_for_dedup = ['title']
    
    # Xử lý và lưu file CORRECT tổng
    if not master_correct_df.empty:
        # Chuẩn hóa title/acronym và loại bỏ các dòng rỗng trước khi final dedup
        master_correct_df['title'] = master_correct_df['title'].astype(str).str.strip()
        if 'acronym' in master_correct_df.columns:
            master_correct_df['acronym'] = master_correct_df['acronym'].astype(str).str.strip()
            subset_for_dedup_correct = ['title', 'acronym']
        else:
            subset_for_dedup_correct = ['title']

        master_correct_df = master_correct_df[master_correct_df['title'] != '']
        if 'acronym' in master_correct_df.columns:
             master_correct_df = master_correct_df[master_correct_df['acronym'] != '']

        final_correct_df = master_correct_df.drop_duplicates(subset=subset_for_dedup_correct, keep='first')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng đúng (không có ghi chú) vào '{aggregated_correct_path}'.")
    else:
        pd.DataFrame().to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'đúng' nào. Đã tạo file rỗng '{aggregated_correct_path}'.")

    # Xử lý và lưu file RECRAWL tổng
    if not master_recrawl_df.empty:
        # Chuẩn hóa title/acronym và loại bỏ các dòng rỗng trước khi final dedup
        master_recrawl_df['title'] = master_recrawl_df['title'].astype(str).str.strip()
        if 'acronym' in master_recrawl_df.columns:
            master_recrawl_df['acronym'] = master_recrawl_df['acronym'].astype(str).str.strip()
            subset_for_dedup_recrawl = ['title', 'acronym']
        else:
            subset_for_dedup_recrawl = ['title']

        master_recrawl_df = master_recrawl_df[master_recrawl_df['title'] != '']
        if 'acronym' in master_recrawl_df.columns:
             master_recrawl_df = master_recrawl_df[master_recrawl_df['acronym'] != '']
             
        final_recrawl_df = master_recrawl_df.drop_duplicates(subset=subset_for_dedup_recrawl, keep='first')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng cần recrawl (có ghi chú) vào '{aggregated_recrawl_path}'.")
    else:
        pd.DataFrame().to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'cần recrawl' nào. Đã tạo file rỗng '{aggregated_recrawl_path}'.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # 1. Danh sách các file input ban đầu (ví dụ: các file batch_X_check.csv)
    input_csv_files_for_note_check = [
        './src/conference/evaluate/batch4.csv',
        './src/conference/evaluate/batch5.csv',
        './src/conference/evaluate/batch9.csv',
        './src/conference/evaluate/batch14.csv',
        './src/conference/evaluate/batch15.csv',
        './src/conference/evaluate/batch18.csv',
        './src/conference/evaluate/batch20.csv'
    ]

    # 2. Danh sách các file là KẾT QUẢ của lần recrawl trước đó (sau khi đã được sửa)
    # Ví dụ: nếu bạn đã chạy recrawl cho batch4 và nhận được file recrawl_batch_4_result.csv
    recrawled_results_files = [
        './src/conference/evaluate/recrawl_batch_4_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_5_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_9_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_14_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_15_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_18_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_20_check_lan_1.csv',

        './src/conference/evaluate/recrawl_all_tri_1_50.csv',
        './src/conference/evaluate/recrawl_all_tri_51_100.csv',
        './src/conference/evaluate/recrawl_all_tri_101_150.csv',

        './src/conference/evaluate/recrawl_all_tri_1_32_lan_2.csv',

        './src/conference/evaluate/recrawl_all_tri_lan_3.csv',

    ]

    # 3. Thư mục để chứa các file output riêng lẻ
    individual_output_directory_note = './src/conference/evaluate/tri_check_outputs'

    # 4. Đường dẫn cho 2 file tổng hợp cuối cùng
    aggregated_correct_file_note = './src/conference/evaluate/tri_all_correct_final.csv'
    aggregated_recrawl_file_note = './src/conference/evaluate/tri_all_recrawl_final.csv'

    # 5. Gọi hàm xử lý chính
    process_files_by_note(
        input_files=input_csv_files_for_note_check,
        output_dir=individual_output_directory_note,
        aggregated_correct_path=aggregated_correct_file_note,
        aggregated_recrawl_path=aggregated_recrawl_file_note,
        recrawled_files=recrawled_results_files # TRUYỀN THAM SỐ MỚI VÀO ĐÂY
    )

In [2]:
import pandas as pd
import os

def merge_csv_common_columns_ordered(file_paths, output_filename):
    """
    Đọc nhiều file CSV, tìm các cột chung (phân biệt hoa thường),
    sau đó hợp nhất các file chỉ với các cột chung đó.
    Thứ tự các cột trong file kết quả sẽ theo thứ tự của các cột chung
    trong file CSV cuối cùng trong danh sách input.

    Args:
        file_paths (list): Danh sách các đường dẫn đầy đủ đến các file CSV.
        output_filename (str): Đường dẫn và tên file CSV đầu ra.
    """
    if not file_paths:
        print("Lỗi: Danh sách đường dẫn file CSV trống. Không có gì để xử lý.")
        return

    dataframes = []
    # Bước 1: Đọc tất cả các file CSV vào DataFrame
    for fp in file_paths:
        try:
            df = pd.read_csv(fp)
            dataframes.append(df)
            print(f"Đã đọc thành công: {fp} (có {len(df.columns)} cột)")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file tại đường dẫn: {fp}. Bỏ qua file này.")
            continue
        except pd.errors.EmptyDataError:
            print(f"Cảnh báo: File trống hoặc không có header: {fp}. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Lỗi khi đọc file {fp}: {e}. Bỏ qua file này.")
            continue

    if not dataframes:
        print("Không có file CSV nào được đọc thành công. Không thể thực hiện merge.")
        return

    # Bước 2: Tìm danh sách các cột chung có ở tất cả các file
    # Bắt đầu với set các cột của DataFrame đầu tiên
    common_columns_set = set(dataframes[0].columns)

    # Lặp qua các DataFrame còn lại và lấy phần giao của các cột
    for i in range(1, len(dataframes)):
        common_columns_set.intersection_update(set(dataframes[i].columns))

    if not common_columns_set:
        print("Không tìm thấy cột chung nào ở TẤT CẢ các file CSV đã đọc. Không tạo file mới.")
        return

    # Bước 3: Lấy thứ tự cột từ file cuối cùng trong danh sách
    # Lấy DataFrame cuối cùng đã đọc thành công
    last_df = dataframes[-1]
    
    # Lọc ra chỉ những cột chung có trong DataFrame cuối cùng, theo đúng thứ tự của nó
    ordered_common_columns = [col for col in last_df.columns if col in common_columns_set]
    
    # Kiểm tra lại, phòng trường hợp kỳ lạ là file cuối cùng không có tất cả các cột chung
    # Điều này không nên xảy ra nếu common_columns_set được tính đúng
    if len(ordered_common_columns) != len(common_columns_set):
        # Đây là trường hợp rất hiếm, chỉ khi logic common_columns_set bị lỗi
        # hoặc có một file "cuối cùng" rỗng hoặc không có cột.
        # Để an toàn, chúng ta có thể sắp xếp lại theo alphabet nếu cần
        print("Cảnh báo: Thứ tự cột từ file cuối cùng không chứa tất cả các cột chung. Sẽ sắp xếp theo alphabet.")
        ordered_common_columns = sorted(list(common_columns_set))

    print(f"\nTìm thấy {len(ordered_common_columns)} cột chung ở tất cả các file:")
    print(f"Thứ tự cột trong file output sẽ dựa trên file cuối cùng: {file_paths[-1]}")
    print(f"Thứ tự các cột: {ordered_common_columns}")

    # Bước 4: Tạo danh sách các DataFrame mới, chỉ chứa các cột chung theo thứ tự mong muốn
    filtered_dataframes = []
    for df in dataframes:
        # Lọc DataFrame để chỉ giữ lại các cột chung và sắp xếp lại chúng
        # df[ordered_common_columns] sẽ tự động bỏ qua các cột không tồn tại trong df hiện tại
        # nếu có lỗi do common_columns_set được tính sai hoặc df rỗng.
        # Tuy nhiên, pandas sẽ báo lỗi nếu một cột trong ordered_common_columns không có trong df.
        # Chúng ta cần đảm bảo rằng mỗi df đều có tất cả các cột chung.
        current_df_cols = [col for col in ordered_common_columns if col in df.columns]
        
        # Nếu có DataFrame nào không có đủ cột chung, nó sẽ không được thêm vào
        if len(current_df_cols) == len(ordered_common_columns):
            filtered_dataframes.append(df[ordered_common_columns])
        else:
            # Điều này chỉ xảy ra nếu có lỗi trong logic tìm common_columns_set
            # hoặc nếu một file nào đó được đọc thành công nhưng thiếu cột chung
            # mà đã được xác định là chung.
            print(f"Cảnh báo: Một DataFrame ({file_paths[dataframes.index(df)]}) thiếu một số cột chung. Bỏ qua DataFrame này trong quá trình merge.")

    if not filtered_dataframes:
        print("Không có DataFrame nào đủ điều kiện để merge sau khi lọc và sắp xếp cột.")
        return

    # Bước 5: Hợp nhất tất cả các DataFrame đã lọc lại với nhau
    try:
        merged_df = pd.concat(filtered_dataframes, ignore_index=True)

        # Bước 6: Lưu DataFrame đã hợp nhất vào file CSV mới
        merged_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\nĐã hợp nhất thành công các file vào '{output_filename}'.")
        print(f"File kết quả có {merged_df.shape[0]} dòng và {merged_df.shape[1]} cột.")
    except Exception as e:
        print(f"Lỗi khi hợp nhất các DataFrame hoặc ghi file: {e}")

# --- Cách sử dụng ---
if __name__ == "__main__":
    # Đặt danh sách các đường dẫn đến file CSV của bạn vào đây.
    # Đảm bảo file cuối cùng trong danh sách là file bạn muốn lấy thứ tự cột.
    input_files = ['./src/conference/evaluate/ALL_BATCHES_correct_final.csv','./src/conference/evaluate/tri_all_correct_final.csv','./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv']


    # Đặt tên cho file CSV kết quả đầu ra
    output_merged_file = './src/conference/evaluate/full.csv'

    # Gọi hàm để thực hiện việc merge
    print("Bắt đầu quá trình merge các file CSV...")
    merge_csv_common_columns_ordered(input_files, output_merged_file)

    print("\n--- Hoàn thành chương trình ---")

Bắt đầu quá trình merge các file CSV...
Đã đọc thành công: ./src/conference/evaluate/ALL_BATCHES_correct_final.csv (có 34 cột)
Đã đọc thành công: ./src/conference/evaluate/tri_all_correct_final.csv (có 29 cột)
Đã đọc thành công: ./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv (có 25 cột)

Tìm thấy 24 cột chung ở tất cả các file:
Thứ tự cột trong file output sẽ dựa trên file cuối cùng: ./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv
Thứ tự các cột: ['requestId', 'originalRequestId', 'title', 'acronym', 'mainLink', 'cfpLink', 'impLink', 'information', 'conferenceDates', 'year', 'location', 'cityStateProvince', 'country', 'continent', 'type', 'submissionDate', 'notificationDate', 'cameraReadyDate', 'registrationDate', 'otherDate', 'topics', 'publisher', 'summary', 'callForPapers']

Đã hợp nhất thành công các file vào './src/conference/evaluate/full.csv'.
File kết quả có 746 dòng và 24 cột.

--- Hoàn thành chương trình ---
